# Inverse fitting of power weights (Laguerre radii)

Sometimes you do *not* want a purely distance-based partition. Instead, you may know
(or hypothesize) where the **interface** between two sites should lie along the line
connecting them.

In a power/Laguerre tessellation, each site has a weight $w_i$ and the boundary between
sites $i$ and $j$ is defined by equal **power distance**:

$$
\lVert x - p_i\rVert^2 - w_i \;=\; \lVert x - p_j\rVert^2 - w_j.
$$

Along the line segment $p_i \to p_j$, the separating plane intersects at a fraction $t$
(measured from $i$ toward $j$):

$$
t \;=\; \tfrac12 + \frac{w_i - w_j}{2\,d^2}, \qquad d=\lVert p_j - p_i\rVert.
$$

So a desired $t_{ij}$ constrains the **weight difference** $w_i - w_j$.

pyvoro2 provides solvers that fit weights (and corresponding Voro++ radii $r_i=\sqrt{w_i+C}$)
from a list of constraints $(i, j, t_{ij})$.

Important practical note:
a constraint can be “algebraically satisfied” but the pair might still not become a **face** in the
final tessellation (e.g. because a third site blocks it). The result object can optionally report
such inactive constraints (`check_contacts=True`).


In [1]:
import numpy as np
from pprint import pprint

import pyvoro2 as pv


## 1) Two-site example (easy to interpret)

With only two sites, the separating plane is the only interface in the domain.
We ask for $t=0.25$, i.e. the interface is closer to site 0 than to site 1.

Here we also set `r_min=1.0` to choose a radii gauge where the smallest returned radius is 1.0.


In [2]:
points = np.array([[0.0, 0.0, 0.0], [2.0, 0.0, 0.0]], dtype=float)
box = pv.Box(((-5, 5), (-5, 5), (-5, 5)))

constraints = [(0, 1, 0.25)]

fit = pv.fit_power_weights_from_plane_fractions(
    points,
    constraints,
    domain=box,
    t_bounds_mode='none',
    r_min=1.0,
)

print('weights:', fit.weights)
print('radii:', fit.radii)
print('t_target:', fit.t_target)
print('t_pred:', fit.t_pred)


weights: [0. 2.]
radii: [1.         1.73205081]
t_target: [0.25]
t_pred: [0.25]


Now use the fitted radii in an actual power tessellation and inspect the volumes.
(For two points in a box, both cells are always present and share one face.)


In [3]:
cells = pv.compute(
    points,
    domain=box,
    mode='power',
    radii=fit.radii,
    return_faces=True,
    return_vertices=True,
)

print('volumes:', [float(c['volume']) for c in cells])


volumes: [550.0, 449.9999999999999]


## 2) Allowing $t<0$ or $t>1$ and adding penalties

In a power diagram, it is possible for the separating plane to lie **outside** the segment
between the two sites ($t<0$ or $t>1$). This corresponds to one site strongly dominating
the other.

pyvoro2 lets you:

- allow any $t$ values, and
- optionally penalize or forbid predicted $t$ outside a chosen interval.

Two common regimes are:

- **soft bounds**: quadratic penalty when $t$ leaves $[0,1]$ (`t_bounds_mode='soft_quadratic'`)
- **hard bounds**: infeasible outside $[0,1]$ (`t_bounds_mode='hard'`)

You can also add an “avoid the endpoints” exponential penalty to discourage interfaces
too close to $t=0$ or $t=1$ (`t_near_penalty='exp'`).


In [4]:
# An intentionally extreme constraint
constraints2 = [(0, 1, 1.4)]  # plane "behind" site 1 (t > 1)

fit_soft = pv.fit_power_weights_from_plane_fractions(
    points,
    constraints2,
    domain=box,
    t_bounds_mode='soft_quadratic',
    alpha_out=5.0,
    t_near_penalty='exp',
    beta_near=1.0,
    t_margin=0.05,
    r_min=1.0,
)

print('t_target:', fit_soft.t_target)
print('t_pred:', fit_soft.t_pred)
print('warnings:', fit_soft.warnings)


t_target: [1.4]
t_pred: [0.90387053]
warnings: ()


## 3) Multi-site example and contact checking

With multiple sites, a requested pair `(i, j)` might not become adjacent in the final tessellation.
Set `check_contacts=True` to have pyvoro2 compute a tessellation using the fitted radii and report
which constraints correspond to actual faces.

This is valuable when you plan to iterate:
fit → compute tessellation → update constraints/weights.


In [5]:
rng = np.random.default_rng(1)
points3 = rng.uniform(-1.0, 1.0, size=(12, 3))
box3 = pv.Box(((-2, 2), (-2, 2), (-2, 2)))

# Pick a few arbitrary constraints. In a real workflow you would usually choose
# pairs that are expected to be near-neighbors.
constraints3 = [
    (0, 1, 0.45),
    (0, 2, 0.55),
    (3, 4, 0.50),
    (5, 6, 0.60),
]

fit3 = pv.fit_power_weights_from_plane_fractions(
    points3,
    constraints3,
    domain=box3,
    check_contacts=True,
    r_min=0.0,
)

print('rms_residual:', fit3.rms_residual)
print('inactive_constraints:', fit3.inactive_constraints)


rms_residual: 0.0
inactive_constraints: (1,)


You can now use `fit3.radii` in `mode='power'` computations.

If you see many inactive constraints, that does *not* necessarily mean the optimizer failed;
it usually means the requested pair is not a Delaunay neighbor under the fitted weights.


In [6]:
cells3 = pv.compute(points3, domain=box3, mode='power', radii=fit3.radii, include_empty=True)

print('n_cells:', len(cells3))
print('n_empty:', sum(bool(c.get('empty', False)) for c in cells3))


n_cells: 12
n_empty: 0
